# Postgres DataTypes

### Introduction

In this lesson, we'll dig deeper into some of our datatypes to see how we can perform coerce our data just like we might in Python.

### Getting Set Up

To work through this, and following material we'll refer to the pagila database located in the following repository: [pagila data](https://github.com/devrimgunduz/pagila).

To create the database, we can do the following:

1. Clone down the [pagila data](https://github.com/devrimgunduz/pagila) repository
2. Create a new database in postgres (perhaps called `pagila`)
3. Run the `pagila-schema.sql` file against the database to create the necessary tables
4. Run the `pagila-data.sql` file against the database to insert the necessary data

Next, we write a function that returns a connection to the database, as well as the cursor.

> Make sure that the name of the database aligns with the database you created, and that the `user` and `password` is appropriate.

In [5]:
import psycopg2

def get_cursor():
    conn = psycopg2.connect(
    database="pagila",
    user="postgres",
    password="postgres")
    cursor = conn.cursor()
    return conn, cursor

In [6]:
conn, cursor = get_cursor()

And then we can confirm that we can connect to the database by doing an initial query.

In [7]:
cursor.execute('SELECT * FROM store LIMIT 1;')

In [8]:
cursor.fetchall()
# [(1,
#   1,
#   1,
#   datetime.datetime(2020, 2, 15, 4, 57, 12, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=-300, name=None)))]


[]

Ok, now that we have our data setup, let's learn some more about datatypes.

### Going deeper with Datatypes

Now when working with data in postgres, it's a good assumption that any function available to us in Python will also be available to us in postgres.  Now we won't cover these functions extensively, because frankly, there are too many.  

But we should learn a few more functions to develop a sense of just what is available to us in postgres.

Let's start with strings.

### Deeper with Strings

1. Slicing 

Just like with Python, we can slice our strings in Postgres, and we can do so with the `left` function, which returns the leftmost `n` characters of a string.

In [16]:
query = """
SELECT left(first_name, 3) FROM customer LIMIT 5;
"""

In [17]:
cursor.execute(query)
cursor.fetchall()

# [('MAR',), ('PAT',), ('LIN',), ('BAR',), ('ELI',)]

[('MAR',), ('PAT',), ('LIN',), ('BAR',), ('ELI',)]

And we can return the last three letters with the `right` function.

In [76]:
query = """
SELECT right(first_name, 3) FROM customer LIMIT 5;
"""

In [77]:
cursor.execute(query)

In [78]:
cursor.fetchall()

# [('ARY',), ('CIA',), ('NDA',), ('ARA',), ('ETH',)]

[('ARY',), ('CIA',), ('NDA',), ('ARA',), ('ETH',)]

> So `left` and `right` is used to slice our strings from the beginning and end, respectively.

2. Concatenation

Next let's see how we can concatenate two strings in postgres.  We can do so with the concat function.

In [19]:
query = """
SELECT concat(first_name, last_name) FROM customer LIMIT 5;
"""

In [80]:
cursor.execute(query)
cursor.fetchall()

[('MARYSMITH',),
 ('PATRICIAJOHNSON',),
 ('LINDAWILLIAMS',),
 ('BARBARAJONES',),
 ('ELIZABETHBROWN',)]

> So with the `concat` function we can concatenate the values of two different columns.

Let's add a space in there.

In [81]:
query = """
SELECT concat(first_name, ' ',last_name) FROM customer LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[('MARY SMITH',),
 ('PATRICIA JOHNSON',),
 ('LINDA WILLIAMS',),
 ('BARBARA JONES',),
 ('ELIZABETH BROWN',)]

3. Split

Now that we saw how to split combine strings, let's see how to split them apart.  For example, take a look at the addresses.  

> Below address is both the name of the column, and the table in question.

In [20]:
query = """
SELECT address FROM address LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[('47 MySakila Drive',),
 ('28 MySQL Boulevard',),
 ('23 Workhaven Lane',),
 ('1411 Lillydale Drive',),
 ('1913 Hanoi Way',)]

Now, let's just select the middle word from each of the addresses using the `split_part` function. 

In [91]:
query = """
SELECT split_part(address, ' ', 2) FROM address LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[('MySakila',), ('MySQL',), ('Workhaven',), ('Lillydale',), ('Hanoi',)]

> So above we use `split_apart`, the column we are splitting, and the index we are selecting, beginning at the number one (as opposed to zero).

### Working with Dates and TimeStamps

Next, let's work with datetime operators.  Oftentimes we will want to extract a month, day, or year from a date or timestamp.  Let's see an example, where we extract the month.  We'll do so with the customer table.  This is what the customer table looks like.

> <img src="./display_customer.png" width="90%">

So let's see the month that our customers were created.

In [22]:
query = """
SELECT  extract(month from create_date) FROM customer LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[(2.0,), (2.0,), (2.0,), (2.0,), (2.0,)]

> So it looks like customers were created in the month of February.

Now we can extract almost any information we like from a date (day of year, day of week).  For a full list, search for the word `extract` in [the documentation](https://www.postgresql.org/docs/9.1/functions-datetime.html).  Here's another example.

In [102]:
query = """
SELECT  extract(doy from create_date) FROM customer LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()

[(45.0,), (45.0,), (45.0,), (45.0,), (45.0,)]

### Working with Numbers

Now, as you may imagine there is a quite a bit we can do with numeric types as well.  A full list is [here](https://www.postgresql.org/docs/9.1/functions-math.html), but let's explore a few key functions specifically. 

1. `round`

> To see it in action, first we can see the `rental_rate` -- first, without the use of round.

In [4]:
query = """
SELECT rental_rate FROM film LIMIT 2;
"""
cursor.execute(query)
cursor.fetchall()
# [(Decimal('0.99'),), (Decimal('4.99'),)]

[(Decimal('0.99'),), (Decimal('4.99'),)]

And now with `round`.

In [24]:
query = """
SELECT ROUND(rental_rate, 1) FROM film LIMIT 2;
"""
cursor.execute(query)
cursor.fetchall()
# [(Decimal('1.0'),), (Decimal('5.0'),)]

[(Decimal('1.0'),), (Decimal('5.0'),)]

> With `round` the second argument specifies the number of decimal points to round to.

2. modulo

There is also a modulo command in SQL.  For example, let's use it to select every other customer.

In [26]:
conn, cursor = get_cursor()
query = """
SELECT customer_id, first_name FROM customer 
WHERE MOD(customer_id, 2) = 1 LIMIT 5;
"""
cursor.execute(query)
cursor.fetchall()
# [(1, 'MARY'), (3, 'LINDA'), (5, 'ELIZABETH'), (7, 'MARIA'), (9, 'MARGARET')]

[(1, 'MARY'), (3, 'LINDA'), (5, 'ELIZABETH'), (7, 'MARIA'), (9, 'MARGARET')]

* absolute value

Finally we can use `ABS` to calculate the absolute value of an number in postgres.  We do not have any negative values in our database, but we can still show an example.

In [28]:
query = """
SELECT ABS(-5)"""
cursor.execute(query)
cursor.fetchall()

[(5,)]

> So even if we don't have the data, we can still practice by ommitting a `FROM` statement.

### A note on casting 

Now that we know how to work with different datatypes in postgres, let's make sure we know how to convert from one datatype to another.  We do so through casting.

For example, let's use casting to convert the rental rate to an integer.

In [23]:
conn, cursor = get_cursor()

query = """
SELECT title, rental_rate::integer FROM film LIMIT 2;
"""
cursor.execute(query)
cursor.fetchall()

[('ACADEMY DINOSAUR', 1), ('ACE GOLDFINGER', 5)]

As you have guessed the `::` means to convert to a different type -- above from decimal to integer.  Oftentimes, we will use casting to convert to an integer.  But really we can cast to any valid datatype that we choose.   

For example, a good use of casting is to convert into a date or a timestamp.  For example, if we have text values of `'2015-01-01'` or `'01-OCT-2015'` we can convert them into dates.

In [25]:
query = '''
SELECT '01-OCT-2015'::date;
'''
cursor.execute(query)

In [26]:
cursor.fetchall()

[(datetime.date(2015, 10, 1),)]

### Summary

In this lesson, we saw how to work with different datatypes in postgres.  Let's review some of our functions. 

* strings

```SQL
--  slicing
SELECT left(first_name, 3) FROM customer;
-- [('MAR',), ('PAT',), ('LIN',), ('BAR',), ('ELI',)]

-- concatenation
SELECT concat(first_name, ' ',last_name) FROM customer;
-- [('MARYSMITH',), ('PATRICIAJOHNSON',)]

-- splitting data
SELECT split_part(address, ' ', 2) FROM address;
-- [('MySakila',), ('MySQL',)]
```

* datetimes

```SQL
-- extract month
SELECT  extract(month from create_date) FROM customer;
-- [(2.0,), (2.0,), (2.0,), (2.0,), (2.0,)]

-- extract doy
SELECT  extract(doy from create_date) FROM customer;
-- [(45.0,), (45.0,), (45.0,), (45.0,), (45.0,)]
```

* numerical data

```SQL
-- round
SELECT ROUND(rental_rate, 2) FROM film;
-- [(Decimal('1.0'),), (Decimal('5.0'),)]

-- modulo 
SELECT customer_id, first_name FROM customer 
WHERE MOD(customer_id, 2) = 1 LIMIT 5;
-- [(1, 'MARY'), (3, 'LINDA'), (5, 'ELIZABETH')]
```

* casting 

```SQL
SELECT title, rental_rate::integer FROM film LIMIT 2;
-- [('ACADEMY DINOSAUR', 1), ('ACE GOLDFINGER', 5)]
```

### Resources

[String Operators](https://www.postgresql.org/docs/9.1/functions-string.html)

[Datetime Operators](https://www.postgresql.org/docs/9.1/functions-datetime.html)